<h2> IBM Applied Data Science Capstone Project Week 3: Part1</h2>
Aim: Build a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name in Toronto.

# Import the necessary libraries

In [2]:
"""Import libraries"""
import pandas as pd
from bs4 import BeautifulSoup
import requests

C:\Users\thakk\Anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.2) or chardet (2.0.3) doesn't match a supported version!
  RequestsDependencyWarning)


# Create url and send request with get option to scrape the text data

In [4]:
#Send the request with given url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url).text


# Convert the text result into html using BeautifulSoup

In [5]:
#Parse data in html format using BeautifulSoup
soup = BeautifulSoup(result, 'html.parser')

# Find table rows and parse through each row to get Postal Code, Borough and Neighborhood and store in the list

In [6]:
#As there are three columns in data, create three lists to store postal codes, Borough and Neighborhood
postalcode, borough, neighborhood = [], [], []

In [7]:
#Get the data using BeautifulSoup

#Find all the rows
rows = soup.find('table').find_all('tr')

#For each row, find the data in the row
for row in rows:
    cell = row.find_all('td')
    
    if len(cell) > 0:
        postalcode.append(cell[0].text)
        borough.append(cell[1].text)
        neighborhood.append(cell[2].text.rstrip('\n'))

# Create a DataFrame with three columns: PostalData, Borough and Neighborhood

In [8]:

#Create a new dataframe with three columns
toronto_df = pd.DataFrame({'PostalCode': postalcode,
                           'Borough' : borough,
                           'Neighborhood': neighborhood})
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# Drop the rows with neighborhood as 'Not Assigned'

In [9]:
#Ignoring the rows with Neighborhood as Not assigned
toronto_df = toronto_df[toronto_df['Neighborhood'] != 'Not assigned'].reset_index(drop = True)
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


# Group the data with neighborhood in same Borough

In [10]:
#Group the neighborhood in same borough
toronto_df_grouped = toronto_df.groupby(['PostalCode', 'Borough'], as_index = False).agg(lambda x: ','.join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# For not assigned neighborhood, make it same as corresponding Borough

In [11]:
#For a Not assigned neighborhood, making it same as borough
for index, row in toronto_df_grouped.iterrows():
    if row['Neighborhood'] == 'Not Assigned':
        row['Neighborhood'] = row['Borough']
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Print the shape of the resultant dataframe

In [12]:
#Print the shape of the dataframe
toronto_df_grouped.shape

(103, 3)